In [5]:
import requests

# Define the OpenDota API base URL
OPENDOTA_API_URL = "https://api.opendota.com/api/matches/"

# Function to get match data from OpenDota
def get_match_data(match_id):
    url = f"{OPENDOTA_API_URL}{match_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching match data:", response.status_code)
        return None

# Function to analyze team fights and annotate relevant details
def analyze_team_fights(teamfights, streamer_hero_id):
    print("\n--- Team Fight Analysis ---")
    for i, fight in enumerate(teamfights):
        start = fight['start']
        end = fight['end']
        
        # Calculate total kills and damage for each team
        radiant_kills = sum(1 for p in fight['players'][:5] if p.get('deaths', 0) > 0)  # Radiant players
        dire_kills = sum(1 for p in fight['players'][5:] if p.get('deaths', 0) > 0)    # Dire players
        radiant_damage = sum(p.get('damage_done', 0) for p in fight['players'][:5])
        dire_damage = sum(p.get('damage_done', 0) for p in fight['players'][5:])

        # Determine which team likely won the fight
        if radiant_kills > dire_kills:
            winner = "Radiant"
        elif dire_kills > radiant_kills:
            winner = "Dire"
        else:
            winner = "Tie"  # if kills are equal, we could also consider damage

        # Streamer-specific actions with safe check for 'hero_id'
        streamer_data = next((p for p in fight['players'] if p.get('hero_id') == streamer_hero_id), None)
        if streamer_data:
            streamer_damage = streamer_data.get('damage_done', 0)
            streamer_deaths = streamer_data.get('deaths', 0)
            streamer_kills = streamer_data.get('kills', 0)
        else:
            streamer_damage = 0
            streamer_deaths = 0
            streamer_kills = 0

        # Print the analysis for this team fight
        print(f"Team Fight {i + 1} - Start: {start}s, End: {end}s")
        print(f"  Winner: {winner}")
        print(f"  Radiant Kills: {radiant_kills}, Dire Kills: {dire_kills}")
        print(f"  Radiant Damage: {radiant_damage}, Dire Damage: {dire_damage}")
        print(f"  Streamer (Hero ID {streamer_hero_id}) Actions:")
        print(f"    Damage Done: {streamer_damage}")
        print(f"    Kills: {streamer_kills}")
        print(f"    Deaths: {streamer_deaths}\n")

# Main function to run the code
def main():
    match_id = "7937741085"  # Replace with your Match ID
    streamer_hero_id = 94     # Replace with your streamer's hero ID in this match
    
    match_data = get_match_data(match_id)
    
    if match_data:
        # Extract and analyze team fights
        if 'teamfights' in match_data:
            analyze_team_fights(match_data['teamfights'], streamer_hero_id)
        else:
            print("No team fights data available.")

# Run the main function
if __name__ == "__main__":
    main()



--- Team Fight Analysis ---
Team Fight 1 - Start: 717s, End: 757s
  Winner: Radiant
  Radiant Kills: 3, Dire Kills: 0
  Radiant Damage: 0, Dire Damage: 0
  Streamer (Hero ID 94) Actions:
    Damage Done: 0
    Kills: 0
    Deaths: 0

Team Fight 2 - Start: 1537s, End: 1585s
  Winner: Radiant
  Radiant Kills: 3, Dire Kills: 2
  Radiant Damage: 0, Dire Damage: 0
  Streamer (Hero ID 94) Actions:
    Damage Done: 0
    Kills: 0
    Deaths: 0

Team Fight 3 - Start: 2056s, End: 2106s
  Winner: Dire
  Radiant Kills: 1, Dire Kills: 3
  Radiant Damage: 0, Dire Damage: 0
  Streamer (Hero ID 94) Actions:
    Damage Done: 0
    Kills: 0
    Deaths: 0

Team Fight 4 - Start: 2156s, End: 2197s
  Winner: Radiant
  Radiant Kills: 2, Dire Kills: 1
  Radiant Damage: 0, Dire Damage: 0
  Streamer (Hero ID 94) Actions:
    Damage Done: 0
    Kills: 0
    Deaths: 0

Team Fight 5 - Start: 2635s, End: 2666s
  Winner: Tie
  Radiant Kills: 2, Dire Kills: 2
  Radiant Damage: 0, Dire Damage: 0
  Streamer (Hero ID 

In [6]:
import requests
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext

# Define the OpenDota API base URL
OPENDOTA_API_URL = "https://api.opendota.com/api/matches/"

# Function to get match data from OpenDota
def get_match_data(match_id):
    url = f"{OPENDOTA_API_URL}{match_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching match data:", response.status_code)
        return None

# Function to annotate events based on match data
def annotate_events(match_data, streamer_hero_id):
    annotations = []
    
    # Annotate Team Fights
    if 'teamfights' in match_data:
        annotations.append("=== Team Fights ===")
        for fight in match_data['teamfights']:
            start = fight['start']
            end = fight['end']
            # Calculate kills and damage by team
            radiant_kills = sum(1 for p in fight['players'][:5] if p.get('deaths', 0) > 0)
            dire_kills = sum(1 for p in fight['players'][5:] if p.get('deaths', 0) > 0)
            radiant_damage = sum(p.get('damage_done', 0) for p in fight['players'][:5])
            dire_damage = sum(p.get('damage_done', 0) for p in fight['players'][5:])
            
            # Determine outcome
            winner = "Radiant" if radiant_kills > dire_kills else "Dire" if dire_kills > radiant_kills else "Draw"
            annotation = f"Team Fight - Start: {start}s, End: {end}s, Winner: {winner} (Radiant Kills: {radiant_kills}, Dire Kills: {dire_kills})"
            
            # Streamer-specific data
            streamer_data = next((p for p in fight['players'] if p.get('hero_id') == streamer_hero_id), None)
            if streamer_data:
                annotation += f", Streamer: Damage={streamer_data.get('damage_done', 0)}, Kills={streamer_data.get('kills', 0)}, Deaths={streamer_data.get('deaths', 0)}"
            annotations.append(annotation)
    
    # Annotate Roshan Kills
    if 'objectives' in match_data:
        annotations.append("\n=== Roshan Kills ===")
        for obj in match_data['objectives']:
            if obj['type'] == 'CHAT_MESSAGE_ROSHAN_KILL':
                team = "Radiant" if obj.get('key') == "npc_dota_roshan_kill_radiant" else "Dire"
                annotations.append(f"Roshan Kill at {obj['time']}s by {team}")
    
    # Annotate Streamer's Overall Stats
    if 'players' in match_data:
        for player in match_data['players']:
            if player['hero_id'] == streamer_hero_id:
                kills = len(player.get('kills_log', []))
                deaths = player.get('deaths', 0)
                assists = player.get('assists', 0)
                annotations.append(f"\n=== Streamer Stats ===\nKills: {kills}, Deaths: {deaths}, Assists: {assists}")
                break
    
    return annotations

# Fetch and display annotations in the GUI
def fetch_and_display():
    match_id = entry_match_id.get()
    if not match_id.isdigit():
        messagebox.showerror("Invalid Input", "Please enter a valid numeric match ID.")
        return

    match_data = get_match_data(match_id)
    if not match_data:
        messagebox.showerror("Error", "Failed to retrieve match data.")
        return
    
    streamer_hero_id = int(entry_hero_id.get())
    annotations = annotate_events(match_data, streamer_hero_id)
    display_annotations(annotations)

def display_annotations(annotations):
    # Clear previous text
    txt_output.config(state=tk.NORMAL)
    txt_output.delete(1.0, tk.END)
    # Display each annotation
    for annotation in annotations:
        txt_output.insert(tk.END, annotation + "\n")
    txt_output.config(state=tk.DISABLED)

# Tkinter GUI Setup
root = tk.Tk()
root.title("Dota 2 Match Data Annotator")
root.geometry("600x450")
root.configure(bg="#f0f0f0")

# Title Label
title_label = tk.Label(root, text="Dota 2 Match Data Annotator", font=("Helvetica", 16, "bold"), bg="#f0f0f0")
title_label.pack(pady=10)

# Input Frame for Match ID and Hero ID
frame_inputs = tk.Frame(root, bg="#f0f0f0")
frame_inputs.pack(pady=10)

tk.Label(frame_inputs, text="Match ID:", font=("Helvetica", 12), bg="#f0f0f0").grid(row=0, column=0, padx=5, pady=5)
entry_match_id = tk.Entry(frame_inputs, font=("Helvetica", 12), width=15)
entry_match_id.grid(row=0, column=1, padx=5, pady=5)
entry_match_id.insert(0, "7937741085")  # Optional default value for testing

tk.Label(frame_inputs, text="Streamer Hero ID:", font=("Helvetica", 12), bg="#f0f0f0").grid(row=1, column=0, padx=5, pady=5)
entry_hero_id = tk.Entry(frame_inputs, font=("Helvetica", 12), width=15)
entry_hero_id.grid(row=1, column=1, padx=5, pady=5)
entry_hero_id.insert(0, "23")  # Optional default value for testing

# Fetch Button
fetch_button = tk.Button(root, text="Fetch Data", font=("Helvetica", 12, "bold"), bg="#007acc", fg="white", command=fetch_and_display)
fetch_button.pack(pady=10)

# Scrollable Output Text Area
txt_output = scrolledtext.ScrolledText(root, font=("Helvetica", 10), width=70, height=15, wrap=tk.WORD, state=tk.DISABLED)
txt_output.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()


In [4]:
import requests

def get_streamer_hero_id(match_id, account_id):
    url = f"https://api.opendota.com/api/matches/{match_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        match_data = response.json()
        
        # Find the player's entry based on account ID
        for player in match_data['players']:
            if player.get('account_id') == account_id:
                return player['hero_id']
    
    print("Error: Unable to find hero ID.")
    return None

# Example usage
match_id = 7937741085  # Replace with the match ID
account_id = 141690233  # Replace with the streamer's account ID
hero_id = get_streamer_hero_id(match_id, account_id)
print("Streamer's Hero ID:", hero_id)

Streamer's Hero ID: 23
